In [5]:
from transformers import AlbertTokenizer, AlbertForQuestionAnswering
import torch

In [6]:
tokenizer = AlbertTokenizer.from_pretrained('twmkn9/albert-base-v2-squad2')
model = AlbertForQuestionAnswering.from_pretrained('twmkn9/albert-base-v2-squad2')
# inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
# start_positions = torch.tensor([1])
# end_positions = torch.tensor([3])
# outputs = model(**inputs, start_positions=start_positions, end_positions=end_positions)
# loss, start_scores, end_scores = outputs[:3]


In [105]:
question = "Which new peso notes were the highest created by 1944?"
answer_text = 'The first issue in 1942 consisted of denominations of 1, 5, 10 and 50 centavos and 1, 5, and 10 Pesos. The next year brought "replacement notes" of the 1, 5 and 10 Pesos while 1944 ushered in a 100 Peso note and soon after an inflationary 500 Pesos note. In 1945, the Japanese issued a 1,000 Pesos note. This set of new money, which was printed even before the war, became known in the Philippines as Mickey Mouse money due to its very low value caused by severe inflation. Anti-Japanese newspapers portrayed stories of going to the market laden with suitcases or "bayong" (native bags made of woven coconut or Corypha leaf strips) overflowing with the Japanese-issued bills. In 1944, a box of matches cost more than 100 Mickey Mouse pesos. In 1945, a kilogram of camote cost around 1000 Mickey Mouse pesos. Inflation plagued the country with the devaluation of the Japanese money, evidenced by a 60% inflation experienced in January 1944.'


In [106]:
input_ids = tokenizer.encode(question, answer_text)
print('The input has a total of {:} tokens.'.format(len(input_ids)))


The input has a total of 235 tokens.


In [107]:
tokens = tokenizer.convert_ids_to_tokens(input_ids)

# For each token and its id...
for token, id in zip(tokens, input_ids):
    
    # If this is the [SEP] token, add some space around it to make it stand out.
    if id == tokenizer.sep_token_id:
        print('')
    
    # Print the token string and its ID in two columns.
    print('{:<12} {:>6,}'.format(token, id))

    if id == tokenizer.sep_token_id:
        print('')

[CLS]             2
▁which           56
▁new             78
▁pes         10,457
o               111
▁notes        2,434
▁were            46
▁the             14
▁highest      1,554
▁created        679
▁by              34
▁1944         2,921
?                60

[SEP]             3

▁the             14
▁first           64
▁issue        1,513
▁in              19
▁1942         3,148
▁consisted    3,597
▁of              16
▁denominations 19,515
▁of              16
▁1              137
,                15
▁5              331
,                15
▁10             332
▁and             17
▁50           1,222
▁cent         5,802
avo          13,884
s                18
▁and             17
▁1              137
,                15
▁5              331
,                15
▁and             17
▁10             332
▁pes         10,457
os              759
.                 9
▁the             14
▁next           328
▁year           159
▁brought        809
▁                13
"                 7
re              

In [108]:
# Search the input_ids for the first instance of the `[SEP]` token.
sep_index = input_ids.index(tokenizer.sep_token_id)

# The number of segment A tokens includes the [SEP] token istelf.
num_seg_a = sep_index + 1

# The remainder are segment B.
num_seg_b = len(input_ids) - num_seg_a

# Construct the list of 0s and 1s.
segment_ids = [0]*num_seg_a + [1]*num_seg_b

# There should be a segment_id for every input token.
assert len(segment_ids) == len(input_ids)

In [109]:
# Run our example through the model.
start_scores, end_scores = model(torch.tensor([input_ids]), # The tokens representing our input text.
                                 token_type_ids=torch.tensor([segment_ids])) # The segment IDs to differentiate question from answer_text


In [110]:
# Find the tokens with the highest `start` and `end` scores.
k = 3 # number of top answers returned
start_scores.detach().numpy()[0][0] = 0
end_scores.detach().numpy()[0][0] = 0
answer_start = torch.topk(start_scores, k, largest=True)
answer_end = torch.topk(end_scores, k, largest=True)

In [111]:
#no overlapping
for index, token_index in enumerate(answer_start[1][0]):   
    if answer_end[1][0][index] >= answer_start[1][0][index]: #end token after start token
        if index == 0: # first (best) answer
            answer = tokens[answer_start[1][0][index]]
            for i in range(answer_start[1][0][index] + 1, answer_end[1][0][index] + 1):
                if tokens[i][0:1] != '▁':
                    answer += tokens[i]
                else:
                    answer += ' ' + tokens[i]
            print('Answer: "' + answer + '"')
        elif (answer_start[1][0][index] >= answer_end[1][0][index-1]) or (answer_start[1][0][index-1] >= answer_end[1][0][index]):
            # ^start token of current span is after end token of previous span; ^end token of current span is before start of prev
            answer = tokens[answer_start[1][0][index]]
            for i in range(answer_start[1][0][index] + 1, answer_end[1][0][index] + 1):
                if tokens[i][0:1] != '▁':
                    answer += tokens[i]
                else:
                    answer += ' ' + tokens[i]
            print('Answer: "' + answer + '"')

Answer: "▁100"
Answer: "▁500 ▁pesos ▁note"
